In [1]:
# CNN model using ResNet 18 (PHASE 6)

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Initial Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_epochs = 10
learning_rate = 0.0001

# ResNet requires a 224x224 input size, normalize using ImageNet stats
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Loading DataSets
train_dir = r"C:\Users\imran\OneDrive\Robotics Projects\Image Forgery\Image Data\TRAINING_CG-1050\TRAINING"
val_dir = r"C:\Users\imran\OneDrive\Robotics Projects\Image Forgery\Image Data\VALIDATION_CG-1050\VALIDATION"
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Defining Improved Model (ResNet18)
class ResNetBinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.base_model(x).squeeze()

model = ResNetBinaryClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Model
print(" Training Started...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluating the Model
print("\n Evaluation on Validation Set")
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).long()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=["Original", "Tampered"]))

# Save the Improved Model
torch.save(model.state_dict(), "resnet_tampered_patch_classifier.pth")
print("\n Model saved as 'resnet_tampered_patch_classifier.pth'")


C:\Users\imran\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\imran\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 Training Started...
Epoch [1/10], Loss: 0.6873
Epoch [2/10], Loss: 0.5833
Epoch [3/10], Loss: 0.4954
Epoch [4/10], Loss: 0.4057
Epoch [5/10], Loss: 0.3096
Epoch [6/10], Loss: 0.2440
Epoch [7/10], Loss: 0.1830
Epoch [8/10], Loss: 0.1242
Epoch [9/10], Loss: 0.1287
Epoch [10/10], Loss: 0.1342

 Evaluation on Validation Set
              precision    recall  f1-score   support

    Original       0.68      0.84      0.75       314
    Tampered       0.79      0.61      0.69       314

    accuracy                           0.72       628
   macro avg       0.74      0.72      0.72       628
weighted avg       0.74      0.72      0.72       628


 Model saved as 'resnet_tampered_patch_classifier.pth'
